### This file call all predicting methods on GEF2012Com and Hvaler
Use MeasurePerformance.ipynb file to measure performance of produced prediction
**NOTE:** Don't run parallel version of the predict functions, jupyter has trouble with that. Use RunExperiment.R script instead

In [3]:
library(tidyr)
library(dplyr)
#library("doMC")
source("Lib/PredictRandomForest.R")
source("Lib/PredictDSHW.R")
source("Lib/PredictSemiParametricArima.R")
source("Lib/PredictTBATS.R")
source("Lib/PredictAverageARIMABaseline.R")

In [4]:
#Predict for Hvaler
HvalerTrainingFile = "Hvaler/training_set.csv"
HvalerCompleteFile = "Hvaler/imputed_complete.csv"
OutputDir = "Hvaler/Predictions/"
HvalerClasses = c('POSIXct', rep("numeric", 21))
NCores = 8
Zones = paste0("subs.", seq(5, 5))#Only test 1 zone now
Temperatures = c("T01")
Horizons = seq(1, 24)
trainingDf = read.csv(HvalerTrainingFile, stringsAsFactors=FALSE, colClasses=HvalerClasses)
completeDf = read.csv(HvalerCompleteFile, stringsAsFactors=FALSE, colClasses=HvalerClasses)

In [ ]:
predictAverageARIMABaseline(OutputDir, trainingDf, completeDf, Zones, Horizons, plotResult = FALSE)

In [ ]:
#BE CAREFUL: This will take several hours!
predictRandomForest(OutputDir, trainingDf, completeDf, Zones, Temperatures, Horizons, nDataPoints = -1, plotResult = FALSE)

In [ ]:
#This will also take a lot of time
predictDSHW(OutputDir, trainingDf, completeDf, Zones, Horizons, modifiedDSHW=FALSE, plotResult = FALSE)

In [ ]:
#This will also take a lot of time
predictDSHW(OutputDir, trainingDf, completeDf, Zones, Horizons, modifiedDSHW=TRUE, plotResult = FALSE)

In [ ]:
#This will also take a lot of time
predictSemiParametricArima(OutputDir, trainingDf, completeDf, Zones, Temperatures, Horizons, plotResult = FALSE)

In [ ]:
#This will also take a lot of time
predictTBATS(OutputDir, trainingDf, completeDf, Zones, Horizons, plotResult = FALSE)

In [4]:
#Predict for GEFCom2012
GEFCom2012TrainingFile = "GEFCom2012/training_set.csv"
GEFCom2012CompleteFile = "GEFCom2012/complete.csv"
OutputDir = "GEFCom2012/Predictions/"
GEFCom2012Classes = c('POSIXct', rep("numeric", 32))
Zones = paste0("zone.", seq(14, 14))
Temperatures = c("T01","T02","T03","T04","T05","T06","T07","T08","T09","T10","T11")
Horizons = seq(1, 24)
trainingDf = read.csv(GEFCom2012TrainingFile, stringsAsFactors=FALSE, colClasses=GEFCom2012Classes)
completeDf = read.csv(GEFCom2012CompleteFile, stringsAsFactors=FALSE, colClasses=GEFCom2012Classes)

In [12]:
predictAverageARIMABaselineParallel(OutputDir, trainingDf, completeDf, Zones, Horizons, plotResult = FALSE, saveResult=TRUE)

In [ ]:
#This will take several hours!
predictRandomForest(OutputDir, trainingDf, completeDf, Zones, Temperatures, Horizons, nDataPoints = -1, plotResult = FALSE)

In [ ]:
predictDSHW(OutputDir, trainingDf, completeDf, Zones, Horizons, modifiedDSHW=FALSE, plotResult = FALSE)

In [ ]:
predictDSHW(OutputDir, trainingDf, completeDf, Zones, Horizons, modifiedDSHW=TRUE, plotResult = FALSE)

In [13]:
predictSemiParametricArimaParallel(OutputDir, trainingDf, completeDf, Zones, Temperatures, Horizons, plotResult = FALSE)

In [ ]:
predictTBATSParallel(OutputDir, trainingDf, completeDf, Zones, Horizons, plotResult = FALSE, saveResult=FALSE)

In [ ]:
install.packages("bbemkr")

In [ ]:
install.packages("ranger")

In [8]:
stopifnot(require('KernSmooth'))
    stopifnot(require('bbemkr'))
    stopifnot(require('mgcv'))
    stopifnot(require('timeDate'))
    stopifnot(require("MASS"))
    stopifnot(require('forecast'))
    stopifnot(require('dplyr'))
    stopifnot(require('lubridate'))
    stopifnot(require('xts'))
    
    source("Lib/SavePredictions.R")
    #Setup loging file
    source("Lib/SetupLog.R")
    horizons = Horizons
    zones = Zones
    temperatures = Temperatures
    #Identify where are the start and end of the prediction periods by shifting index of NA
    idxNaCases = !complete.cases(trainingDf)
    startPoints =  which(idxNaCases & !c(FALSE, head(idxNaCases, -1)) & c(tail(idxNaCases, -1), TRUE))
    endPoints = which(idxNaCases & c(TRUE, head(idxNaCases, -1)) & !c(tail(idxNaCases, -1), FALSE))
    startDates = trainingDf$DateTime[startPoints]
    endDates = trainingDf$DateTime[endPoints]
    nTestingPeriods = length(startDates)

    #Create Time Features
    startYear = year(completeDf$DateTime[1])
    endYear = year(tail(completeDf$DateTime, 1))
    years = seq(startYear, endYear)
    NorwayHolidays = c(EasterMonday(years), 
                                  Ascension(years), 
                                  PentecostMonday(years), 
                                  LaborDay(years), 
                                  GoodFriday(years), 
                                  BoxingDay(years), 
                                  GoodFriday(years)-86400);
    completeDf = completeDf %>% mutate(Holiday= isHoliday(as.timeDate(DateTime), NorwayHolidays, wday=0:6)) %>%
                                mutate(ChristmasDay= isHoliday(as.timeDate(DateTime), ChristmasDay(years), wday=0:6)) %>%
                                mutate(ChristmasEve= isHoliday(as.timeDate(DateTime), ChristmasEve(years), wday=0:6)) %>%
                                mutate(NewYearsDay= isHoliday(as.timeDate(DateTime), NewYearsDay(years), wday=0:6)) %>%
                                mutate(DoW = factor(wday(DateTime))) %>%
                                mutate(ToY = as.numeric(strftime(DateTime, format = "%j"))
                                            +as.numeric(strftime(DateTime, format="%H"))/24)


    #Assume the zone and temperature here
    maxHorizon = max(horizons)
    predictions = rep(list(trainingDf), maxHorizon);
    for (zone in zones){
        #Find the best correlated temperature with current zone
        maxCor = -1
        bestTemp = temperatures[[1]]
        for (temp in temperatures){
            correlation = cor(completeDf[[zone]], completeDf[[temp]])
            if (correlation > maxCor){
                maxCor = correlation
                bestTemp = temp
            }
        }
        completeDf$T = completeDf[[bestTemp]]
        featureDf = completeDf %>% dplyr::select(one_of(c("DateTime", 
                                            "Holiday", 
                                            "ChristmasDay", 
                                            "ChristmasEve", 
                                            "NewYearsDay", 
                                            "DoW", "ToY", "T", zone )))
        featureDf$E = featureDf[[zone]]
        featureDf$SmoT = featureDf$T
        for (i in 2:length(featureDf$T)){
            featureDf$SmoT[i] = 0.15*featureDf$T[i] + 0.85*featureDf$SmoT[i-1]
        }

        featureDf$LT = rep(0, nrow(featureDf))
        featureDf$Residuals = rep(0, nrow(featureDf)) #This is used to train an Arima on residuals
        for (period in seq(1, nTestingPeriods)){
            startTime = Sys.time()
            
            #Prepare for training model
            startDate = startDates[period]
            endDate = endDates[period]
            trainData = featureDf %>% filter(DateTime < startDate)
            trainData$LT = longTermTrend(trainData$E, trainData$T, trainData$DateTime)

            spec = E ~ LT + s(T) + s(SmoT) + s(ToY,bs="cc", k = 100) + 
                        DoW + ChristmasDay + ChristmasEve + NewYearsDay + Holiday
            for (hour in 0:23){ #different model for each hour
                #Becareful with logical vector index that not has the same length with dataframe, use which()
                trainingIdx = which(hour(trainData$DateTime)==hour) 
                trainDataAtH = trainData[trainingIdx, ]

                model = gam(spec, data=trainDataAtH)
                #gam.check(model)

                #Testing
                testingIdx = which((hour(featureDf$DateTime)==hour) & (featureDf$DateTime >= startDate) & (featureDf$DateTime <= endDate))
                featureDf$LT[testingIdx] = rep(tail(trainData$LT, 1), length(testingIdx))
                testData = featureDf[testingIdx, ]
                prediction = predict(model, testData)
                for (h in horizons){ #prediction is the same for all
                    predictions[[h]][testingIdx, zone] = prediction
                }
                featureDf$Residuals[trainingIdx] = model$model$E - model$fitted.values #residuals used for training arima
                featureDf$Residuals[testingIdx] = featureDf$E[testingIdx] - prediction #residuals used for testing arima
            }
            #Run Arima here
            startPoint = startPoints[period]
            endPoint = endPoints[period]
            startTrainingPoint = startPoint - 12*7*24 #Only get 3 month of data for training
            xts = xts(featureDf$Residuals, featureDf$DateTime)
            trainXts = xts[startTrainingPoint:(startPoint-1)]
                        
            model = auto.arima(as.ts(trainXts), seasonal=TRUE)
            #model = Arima(trainXts, order = c(3, 0, 3), seasonal=list(order=c(3, 0, 3), period = 24))
            testXts = trainXts
            for (currentPoint in seq(startPoint, endPoint)){
                refit = Arima(as.ts(testXts), model=model)
                prediction = forecast(refit, h=maxHorizon)$mean
                for (h in horizons){
                    if (currentPoint+h-1 <= endPoint){
                       predictions[[h]][currentPoint+h-1, zone] = predictions[[h]][currentPoint+h-1, zone] + prediction[h]
                    }
                }            
                testXts = c(testXts, xts[currentPoint])
            }
            
            prettyPrint(paste0("semiParametric|", zone, "|period ", period, "|Done in ", 
                               as.numeric(Sys.time()-startTime, units = "secs")));
        }  
    }

[1] "2016-11-27 00:07:14 1311cc8c0fb3-6447: semiParametric|zone.14|period 1|Done in 84.9470386505127"
[1] "2016-11-27 00:08:32 1311cc8c0fb3-6447: semiParametric|zone.14|period 2|Done in 77.8861882686615"
[1] "2016-11-27 00:09:38 1311cc8c0fb3-6447: semiParametric|zone.14|period 3|Done in 65.56134557724"
[1] "2016-11-27 00:11:24 1311cc8c0fb3-6447: semiParametric|zone.14|period 4|Done in 106.720576524734"


In [5]:
longTermTrend <- function(E, T, DateTime, BANDWIDTH=12){
    E.xts = xts(E, DateTime)
    T.xts = xts(T, DateTime)
    E.month.xts = apply.monthly(E.xts, FUN="mean", na.rm = TRUE)
    I = as.numeric(format(index(E.month.xts), "%m"))
    #I = quarters (index(E.month.xts), "%m")
    if (length(I) > 24){
        I = factor(I)
    }
    T = apply.monthly(T.xts[index(E.xts)], FUN="mean", na.rm = TRUE)
    E.model = gam(E.month.xts ~ I + s(T))#ERROR!!
    E.est = E.model$fitted.values
    E.residuals = E.model$residuals
    a = NadarayaWatsonkernel(1:length(E.residuals), E.residuals, BANDWIDTH, 1:length(E.residuals))
    a$mh
    month = as.numeric(format(index(E.xts), "%m"))
    year = as.numeric(format(index(E.xts), "%y"))

    idx = (year-year[1])*12+(month-month[1])+1

    days = rep(0, length(E.xts))
    for (i in 1:length(a$mh)){
        days[which(idx == i)]  = sum(idx==i)
    }

    fraction = as.numeric(format(index(E.xts), "%d"))*24 + as.numeric(format(index(E.xts), "%H"))

    trend.month = c(a$mh[1]-(a$mh[2]-a$mh[1]), a$mh)
    trend = rep(0, length(E.xts))
    trend = (trend.month[idx+1]-trend.month[idx])*fraction/days + trend.month[idx]
    trend.xts = xts(trend, order.by = index(E.xts))
    return(drop(coredata(trend.xts)))
}